### This is a computational acoustic tutorial based on Finite Element Method (FEM) by using in-house developed Python code


Let consider an acoustic wave propagation problem in two-media domain. 

The first medium is a fluid (air) and the second one is a porous material which is able to absorb the sound wave. The left side is subjected to a unit velocity excitation and a rigid wall is bounded the right side of the domain. 

This test case is a very typical kundlt tube measurement set-up. The schematic of the problem is shown in the following figure:

![problem setup](figs/prob_1d_1.png)

Let import the required libraries and FEM modules

In [49]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('/home/shaoqi/Devlop/PyXfem/PyAcoustiX/')  # add the path of PyAcoustiX


Set the problem parameters and material properties

In [50]:
from fem.materials import Air, EquivalentFluid

air = Air('classical air')  # create an air object with name
# cporous material properties
phi          = 0.98  # porosity
sigma        = 3.75e3 # resistivity
alpha        = 1.17  # Tortuosity
Lambda_prime = 742e-6  # Viscous characteristic length
Lambda       = 110e-6  #  thermal characteristic length
foam = EquivalentFluid('foam', phi, sigma, alpha, Lambda_prime, Lambda)  # create a porous material object with name

Excitation frequency

In [51]:
frequency = 1000  # Hz
omega = 2 * np.pi * frequency  # angular frequency

Mesh and two-media domain definition

In [52]:
from fem.mesh import Mesh1D  # import 1D mesh class

num_elem = 200  # number of elements
num_nodes = num_elem + 1  # number of nodes
nodes = np.linspace(-1, 1, num_nodes)

elem_connec1 = np.arange(0, num_elem)
elem_connec2 = np.arange(1, num_nodes)
connectivity = np.vstack((elem_connec1, elem_connec2)).T
# crate 1D mesh with nodes coordinates and connectivity
mesh = Mesh1D(nodes, connectivity)
mesh.get_mesh()  # get mesh information: element number: coordinates
# mesh.get_nodes_from_elem()[1]

AttributeError: 'Mesh1D' object has no attribute 'get_nodes_from_elem'

In [ ]:
# define subdomains: domain physic (material) and the elements in the domain
air_elements  = np.arange(0, int(num_elem/2))  # half of the elements are air
foam_elements = np.arange(int(num_elem/2), num_elem)  # half of the elements are foam
subdomains = {air: air_elements, foam: foam_elements}  # define subdomains
# we then need to check if these two materials (physic) are compatible in the domain
from fem.utilities import check_material_compability
check_material_compability(subdomains)

Great ! materials are compatible with each other and the subdoamin is well defined.

Let's construct the our FEM bases on the mesh (domain)

In [ ]:
# firt import the basis (element) class
from fem.basis import Lobbato1DElement
print(mesh.get_nodes_from_elem(0))  # get the nodes coordinates of the first element
elements2node = mesh.get_mesh()  #  get mesh information to be used to construct the bases
# In the AcoustiX, the basis can be constructed element-wise
order = 2  # order of the shape functions (globally): make sure we have sufficient dofs per wave length
Pf_bases = []
for mat, elems in subdomains.items():
    if mat.TYPE == 'Fluid':
        Pf_bases += [Lobbato1DElement('Pf', order, mesh.get_nodes_from_elem(elem)) for elem in elems]  # arguments: name, order, nodes coordinates

In [ ]:
# the bases , mesh and subdomains are used to construct the finite element space
from fem.dofhandler import FESpace

Helmholtz_space = FESpace(mesh, subdomains, Pf_bases)  # args: bases name, bases
print(f"The number of global dofs is {Helmholtz_space.get_nb_dofs()}.")

In [ ]:
# assemle the global matrix, here we are treating the Helmholtz equation, 
# a wrapped Helmholtz assembler in AcoustiX can be directly used
from fem.physic_assembler import HelmholtzAssembler
# args: dof_handler, subdomains, dtype (type of left hand matrix and right hand vector)
Helmholtz_assember = HelmholtzAssembler(Helmholtz_space, subdomains, dtype=np.complex128)  # complex matrix and solution
Helmholtz_assember.assembly_global_matrix(Pf_bases, 'Pf', omega)  # assemble the global matrix with excited frequency
left_hand_matrix = Helmholtz_assember.get_global_matrix()  # get the global matrix
# the global left hand side matrix is construct, let us visualize it's sparse pattern with an AcoustiX utility
from fem.utilities import plot_matrix_partten
plot_matrix_partten(left_hand_matrix)  # display the matrix sparse partten

In [ ]:
# it's time to construct the right hand side vector (boundary conditions)

from fem.BCs_impose import ApplyBoundaryConditions
# define our right hand side vector
right_hand_vec = np.zeros(Helmholtz_assember.nb_global_dofs, dtype=np.complex128)
# as velocity is excited, which a kind of Neumann (natural) boundary condition
nature_bcs = {'type': 'velocity', 'value': 1, 'position': -1}
BCs_applier = ApplyBoundaryConditions(mesh, Helmholtz_space, left_hand_matrix, right_hand_vec, omega)
BCs_applier.apply_nature_bc(nature_bcs, var='Pf')
print(f"{nature_bcs['type']} boundary condition is applied at x={nature_bcs['position']}.")

In [ ]:
# the last step is to solve the linear system
from fem.solver import LinearSolver
linear_solver = LinearSolver(fe_space=Helmholtz_space)
linear_solver.solve(left_hand_matrix, right_hand_vec)
sol = linear_solver.u

In [ ]:
# to verify the our FEM solution, a reference solution is needed. 
# Here we use the analytical solution which has been implemented in AcoustiX
from analytical.fluid_sol import DoubleleLayerKundltTube
# by giving mesh, media, frequency and boundary conditions, the analytical solution can be constructed
kundlt_tube = DoubleleLayerKundltTube(mesh, air, foam, omega, nature_bcs)
# we need a vector to contain the analytical solution
ana_sol = np.zeros(num_nodes, dtype=np.complex128)  #initialize the analytical solution vector
kundlt_tube.sol_on_nodes(ana_sol, sol_type='pressure')  # we are interested in the pressure solution

In [ ]:
# Now we can compare the FEM solution with the analytical solution by plotting them
from fem.postprocess import PostProcessField

# create a postprocess object
post_processer = PostProcessField(mesh.nodes, rf'1D Helmholtz ({frequency}$Hz$)')
post_processer.plot_sol((np.real(sol), f'FEM ($p={order}$)', 'solid'), (np.real(ana_sol), 'Analytical', 'dashed'))
plt.show()

Voilà, here is our solution and the whole tutorial is done. You can play with the demo code by changing the problem, numerical parameters and see how the solution is changed, for instance:
- Change the properties of the porous material
- Change the excitation frequency
- Change the mesh size (number of elements)
- Change the polynomial order of the FEM bases
- Use different velocity to excite the domain
There are more advanced features that can be applied on this problem, for instance:
- optimize the sparsity pattern by using a method in our fem solver module
- use variable order of the FEM bases on each element
- compute the relative error of the solutions